<a href="https://colab.research.google.com/github/deepkick/kagglebook/blob/master/ch01/PredictFutureSales_Traditional_TS_001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Time series Basics : Exploring traditional TS 
https://www.kaggle.com/jagangupta/time-series-basics-exploring-traditional-ts  

In [1]:
!pwd

/content


In [2]:
!ls ../input/*

../input/item_categories.csv  ../input/sample_submission.csv
../input/items.csv	      ../input/shops.csv
../input/sales_train.csv      ../input/test.csv


In [3]:
# Basic packages
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random as rd # generating random numbers
import datetime # manipulating date formats
# Viz
import matplotlib.pyplot as plt # basic plotting
import seaborn as sns # for prettier plots


# TIME SERIES
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.stattools import adfuller, acf, pacf,arma_order_select_ic
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs


# settings
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
# Import all of them 
sales=pd.read_csv("../input/sales_train.csv")

# settings
import warnings
warnings.filterwarnings("ignore")

item_cat=pd.read_csv("../input/item_categories.csv")
item=pd.read_csv("../input/items.csv")
sub=pd.read_csv("../input/sample_submission.csv")
shops=pd.read_csv("../input/shops.csv")
test=pd.read_csv("../input/test.csv")

In [5]:
sales

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0.0,59.0,22154.0,999.00,1.0
1,03.01.2013,0.0,25.0,2552.0,899.00,1.0
2,05.01.2013,0.0,25.0,2552.0,899.00,-1.0
3,06.01.2013,0.0,25.0,2554.0,1709.05,1.0
4,15.01.2013,0.0,25.0,2555.0,1099.00,1.0
...,...,...,...,...,...,...
1225263,14.12.2013,11.0,55.0,492.0,600.00,4.0
1225264,30.12.2013,11.0,55.0,492.0,600.00,3.0
1225265,29.12.2013,11.0,55.0,492.0,600.00,4.0
1225266,28.12.2013,11.0,55.0,492.0,600.00,2.0


In [6]:
item_cat

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4
...,...,...
79,Служебные,79
80,Служебные - Билеты,80
81,Чистые носители (шпиль),81
82,Чистые носители (штучные),82


In [7]:
item

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40
...,...,...,...
22165,"Ядерный титбит 2 [PC, Цифровая версия]",22165,31
22166,Язык запросов 1С:Предприятия [Цифровая версия],22166,54
22167,Язык запросов 1С:Предприятия 8 (+CD). Хрустале...,22167,49
22168,Яйцо для Little Inu,22168,62


In [8]:
sub

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5
...,...,...
214195,214195,0.5
214196,214196,0.5
214197,214197,0.5
214198,214198,0.5


In [9]:
shops

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4
5,"Вологда ТРЦ ""Мармелад""",5
6,"Воронеж (Плехановская, 13)",6
7,"Воронеж ТРЦ ""Максимир""",7
8,"Воронеж ТРЦ Сити-Парк ""Град""",8
9,Выездная Торговля,9


In [10]:
test

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268
...,...,...,...
214195,214195,45,18454
214196,214196,45,16188
214197,214197,45,15757
214198,214198,45,19648


In [13]:
# 行・列ごとに欠損値の個数をカウント
print(sales.isnull().sum())
print(sales.isnull().sum(axis=1))

date              0
date_block_num    1
shop_id           1
item_id           1
item_price        1
item_cnt_day      1
dtype: int64
0          0
1          0
2          0
3          0
4          0
          ..
1225263    0
1225264    0
1225265    0
1225266    0
1225267    5
Length: 1225268, dtype: int64


In [15]:
# リストワイズ削除
sales = sales.dropna()

# 行・列ごとに欠損値の個数をカウント
print(sales.isnull().sum())
print(sales.isnull().sum(axis=1))

date              0
date_block_num    0
shop_id           0
item_id           0
item_price        0
item_cnt_day      0
dtype: int64
0          0
1          0
2          0
3          0
4          0
          ..
1225262    0
1225263    0
1225264    0
1225265    0
1225266    0
Length: 1225267, dtype: int64


In [16]:
#formatting the date column correctly
sales.date=sales.date.apply(lambda x:datetime.datetime.strptime(x, '%d.%m.%Y'))
# check
print(sales.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1225267 entries, 0 to 1225266
Data columns (total 6 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   date            1225267 non-null  datetime64[ns]
 1   date_block_num  1225267 non-null  float64       
 2   shop_id         1225267 non-null  float64       
 3   item_id         1225267 non-null  float64       
 4   item_price      1225267 non-null  float64       
 5   item_cnt_day    1225267 non-null  float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 65.4 MB
None


In [17]:
# Aggregate to monthly level the required metrics

monthly_sales=sales.groupby(["date_block_num","shop_id","item_id"])[
    "date","item_price","item_cnt_day"].agg({"date":["min",'max'],"item_price":"mean","item_cnt_day":"sum"})

## Lets break down the line of code here:
# aggregate by date-block(month),shop_id and item_id
# select the columns date,item_price and item_cnt(sales)
# Provide a dictionary which says what aggregation to perform on which column
# min and max on the date
# average of the item_price
# sum of the sales

In [18]:
# take a peak
monthly_sales.head(20)

date            item_price item_cnt_day
                                      min        max       mean          sum
date_block_num shop_id item_id                                              
0.0            0.0     32.0    2013-01-03 2013-01-31      221.0          6.0
                       33.0    2013-01-03 2013-01-28      347.0          3.0
                       35.0    2013-01-31 2013-01-31      247.0          1.0
                       43.0    2013-01-31 2013-01-31      221.0          1.0
                       51.0    2013-01-13 2013-01-31      128.5          2.0
                       61.0    2013-01-10 2013-01-10      195.0          1.0
                       75.0    2013-01-17 2013-01-17       76.0          1.0
                       88.0    2013-01-16 2013-01-16       76.0          1.0
                       95.0    2013-01-06 2013-01-06      193.0          1.0
                       96.0    2013-01-10 2013-01-10       70.0          1.0
                       98.0    2013-01-04 2013-01-31      268.0         25.0
                       111.0   2013-01-17 2013-01-17       89.0          1.0
                       149.0   2013-01-11 2013-01-28       99.0          3.0
                       151.0   2013-01-16 2013-01-16       75.0          1.0
                       153.0   2013-01-09 2013-01-09      258.0          1.0
                       198.0   2013-01-10 2013-01-10      112.0          1.0
                       210.0   2013-01-05 2013-01-25      118.0          2.0
                       282.0   2013-01-04 2013-01-04      109.0          1.0
                       306.0   2013-01-22 2013-01-22       59.0          1.0
                       351.0   2013-01-21 2013-01-21       89.0          1.0